# Generate Bollinger bands using daily dedicated service - Python

### Overview
Bollinger bands were created by John Bollinger, they are envelopes and plotted around the moving average. The width of the bands is based on the standard deviaton of the closing prices from a moving average. 

The purpose of Bollinger Bands is to provide a relative definition of high and low prices of a market. By definition, prices are high at the upper band and low at the lower band.

This sample demonstrates how to request and plot **on-demand** Bollinger bands from a `daily data service`.

### Inputs/outputs
Bollinger bands indicator requires instrument's identifier,date time intervals and number of periods as per inputs and returns data points for the following components:
1. *Middle band* = n-period moving average
2. *Upper band* = Middle band + (k * n-period standard deviation)
3. *Lower band* = Middle band - (k * n-period standard deviation)

Where:
* *n: number of periods*
* *k: factor to apply to the standard deviation value, k=2 as default value*

This sample shows how to request and plot a simple Bollinger bands graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve Bollinger bands daily data from the hosted service. The queried endpoint in this script are:
* *DailyBollingerService*: to directly retrieve bollinger bands daily data reponse from the server

### Modules required
1. Systemathics:
     * *systemathics.apis.services.daily_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matplotlib* as per display package

***

# Run daily Bollinger bands sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly matplotlib

In [ ]:
pip install systemathics.apis==2.32.*

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.daily_analytics.v1.daily_bollinger_pb2 as daily_bollinger
import systemathics.apis.services.daily_analytics.v1.daily_bollinger_pb2_grpc as daily_bollinger_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *daily Bollinger bands* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The Bollinger bands parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument
ticker = "AAPL";
exchange = "XNGS";

#### 3.2 Bollinger bands parameters

In [ ]:
# set length and deviation parameters
length = 100;
deviation = 0.4;

#### 3.3 Time period selection

In [ ]:
# set data interval (we are using Google date time format) 
my_date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2019, month = 1, day = 1),
    end_date = date.Date(year = 2020, month = 12, day = 31)
)

#### 3.4 Request creation
The following code snippets creates *gRPC client*, process daily Bollinger bands request and returns the request reply:

In [ ]:
# generate daily rsi request
request = daily_bollinger.DailyBollingerRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker), 
    length = length,
    deviation = deviation,
    date_interval = my_date_interval
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the daily Bollinger bands service
        service = daily_bollinger_service.DailyBollingerServiceStub(channel)
        
        # Process the daily Bollinger bands request
        response = service.DailyBollinger(
            request = request, 
            metadata = [('authorization', token)])
        
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

### Step 4: Visualize data

#### 4.1 Retrieve Bollinger bands data
In the following code snippets, the daily Bollinger bands reply is reprocessed in a dataframe in order to visualize the results:

In [ ]:
# create bollinger bands list, set nan null values to avoid zero-values plot
length = len(response.data)
uppers, lowers, middles = [None]*length, [None]*length, [None]*length

for i in range(length):
    # get non null bollinger bands values
    upper = response.data[i].upper.value
    middle = response.data[i].middle.value
    lower = response.data[i].lower.value
    
    # store non null bollinger bands values 
    if  upper != 0:
        uppers[i] = upper
    if middle != 0:
        middles[i] = middle
    if lower != 0:
        lowers[i] = lower

In [ ]:
# retrieve dates and daily prices
dates = [datetime(bb.date.year, bb.date.month, bb.date.day) for bb in response.data]
prices = [bb.value for bb in response.data]

In [ ]:
# create a pandas dataframe with dates, prices and bollinger bands
d = {'Date': dates, 'Upper': uppers, 'Middle': middles, 'Lower': lowers, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot Bollinger bands with matplotlib

In [ ]:
import matplotlib.pyplot as plt

# create figure and axis objects with subplots()
fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'Upper', data=df, marker='', color='purple', linewidth=2, label="Upper")
ax.plot('Date', 'Middle', data=df, marker='', color='orange', linewidth=2, label="Middle")
ax.plot('Date', 'Lower', data=df, marker='', color='green', linewidth=2, label="Lower")
ax.plot('Date', 'Price', data=df, marker='', color='blue', linewidth=2, label="Price")

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('Daily Bollinger bands and prices for {0}-{1}: deviation = {2} | length = {3}'.format(ticker,exchange,deviation,length))
plt.show()

#### 4.3 Plot Bollinger bands with plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# add Bollinger bands and daily prices traces
fig.add_trace(go.Scatter(x=df['Date'], y=df['Upper'],mode='lines', name='Upper band'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Middle'], mode='lines', name='Middle band'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Lower'], mode='lines', name='Lower band'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], mode='lines', name='Price'), secondary_y=True)

# add figure title and legend
fig.update_layout(title = 'Bollinger bands and daily prices for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Bollinger band', secondary_y=False)
fig.update_yaxes(title_text = 'Price', secondary_y=True)
fig.show()